In [1]:
# Stanford Cars 원본 데이터 경로로 이동해 후속 셀에서 파일을 읽습니다.
cd ".\stanford_car_raw"

D:\datasets\stanford_car_raw


In [2]:
# 크롭 작업에 필요한 과학 연산/이미지 모듈을 불러옵니다.
import scipy
import os
import numpy as np

In [3]:
# 학습/테스트 주석(mat) 파일 경로를 설정합니다.
train_anno = r".\stanford_car_raw\cars_train_annos.mat"

In [4]:
test_anno = r".\stanford_car_raw\cars_test_annos_withlabels.mat"

In [5]:
overall_anno = r".\stanford_car_raw\cars_annos.mat"

In [6]:
# 주석 파일을 메모리로 로드해 bbox/레이블 정보를 사용합니다.
test_anno = scipy.io.loadmat(test_anno)
train_anno = scipy.io.loadmat(train_anno)

In [7]:
# 주석 파일을 메모리로 로드해 bbox/레이블 정보를 사용합니다.
data_anno = scipy.io.loadmat(overall_anno)

In [8]:
test_anno_content = test_anno['annotations']
train_anno_content = train_anno['annotations']
class_name = data_anno['class_names'][0]

In [9]:
class_name[173] = 'Ram CV Cargo Van Minivan 2012'

In [41]:
class_name[174]

array(['Rolls-Royce Phantom Drophead Coupe Convertible 2012'],
      dtype='<U51')

In [35]:
train_anno_content[0][80]

(array([[10]], dtype=uint8), array([[55]], dtype=uint8), array([[540]], dtype=uint16), array([[271]], dtype=uint16), array([[8]], dtype=uint8), array(['00081.jpg'], dtype='<U9'))

In [36]:
# 크롭된 이미지를 저장할 학습/테스트 폴더 경로를 지정합니다.
root = r".\stanford_car_cropped\train_cropped"
source = r".\stanford_car_raw\cars_train\cars_train"

In [12]:
from PIL import Image 
from tqdm import tqdm

In [38]:
# 학습 세트 주석을 순회하며 클래스별 폴더 생성 후 bbox로 이미지를 크롭합니다.
idx = set()
for i in train_anno_content[0]:
    # 주석 항목에서 bbox 좌표/클래스/파일명을 풀어 클래스 디렉터리를 만들고 크롭 이미지를 저장합니다.
    x1, y1, x2, y2, clas, fname = i
    cls_idx = clas[0][0]
    idx.add(cls_idx)

In [39]:
# 학습 세트 주석을 순회하며 클래스별 폴더 생성 후 bbox로 이미지를 크롭합니다.
for i in train_anno_content[0]:
    # 주석 행에서 bbox 좌표(x1,y1,x2,y2), 클래스 ID, 파일명을 추출합니다.
    x1, y1, x2, y2, clas, fname = i
    cls_idx = clas[0][0]  # matlab 구조체 형태에서 실제 ID 추출
    clas_name = class_name[cls_idx-1][0]  # ID에 대응하는 클래스 이름 문자열
    # 클래스별 폴더 경로를 만들고 존재하지 않으면 생성합니다.
    clas_file = str(cls_idx) +'_'+clas_name  # '001_ClassName' 형식 폴더명
    doc_root = os.path.join(root, clas_file)
    if not os.path.exists(doc_root):
        os.makedirs(doc_root)
    # 원본 이미지 경로를 구성하고 이미지를 로드합니다.
    img_src = os.path.join(source, fname[0])
    img = Image.open(img_src)
    # bbox 좌표로 잘라낸 뒤 448×448 크기로 리사이즈합니다.
    im_cr = img.crop((x1[0][0], y1[0][0], x2[0][0], y2[0][0])).resize((448, 448))
    # 클래스 폴더 안에 원본 파일명으로 저장합니다.
    image_root = os.path.join(doc_root, fname[0])
    im_cr.save(image_root)
    #break


In [13]:
root = r".\stanford_car_cropped\test_cropped"
source = r".\stanford_car_raw\cars_test\cars_test"

In [14]:
# 테스트 세트 주석을 순회하며 동일하게 크롭/리사이즈하여 저장합니다.
for i in tqdm(range(len(test_anno_content[0]))):
    content = test_anno_content[0][i]
    x1, y1, x2, y2, clas, fname = content
    cls_idx = clas[0][0]
    clas_name = class_name[cls_idx-1][0]
    #create class folder
    clas_file = str(cls_idx) +'_'+clas_name
    doc_root = os.path.join(root, clas_file)
    if not os.path.exists(doc_root):
        os.makedirs(doc_root)
    img_src = os.path.join(source, fname[0])
    #print(img_src)
    img = Image.open(img_src)
    im_cr = img.crop((x1[0][0], y1[0][0], x2[0][0], y2[0][0])).resize((448, 448))
    image_root = os.path.join(doc_root, fname[0])
    im_cr.save(image_root)
    #break

100%|██████████| 8041/8041 [01:42<00:00, 78.43it/s] 


In [24]:
class_name[173]

array(['Ram C/V Cargo Van Minivan 2012'], dtype='<U30')

In [79]:
content

(array([[30]], dtype=uint8), array([[52]], dtype=uint8), array([[246]], dtype=uint8), array([[147]], dtype=uint8), array([[181]], dtype=uint8), array(['00001.jpg'], dtype='<U9'))